# DNN with Hyper-Parameters Tuning

In [9]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "-1" # disable GPU
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation,Dropout

#hyper-parameter tuning imports
from kerastuner.tuners import RandomSearch
from kerastuner.engine.hyperparameters import HyperParameters
import time
LOG_DIR = f"{int(time.time())}" # folder names as a timestamp

SEED = 111 # constant seed for reproducibility
os.environ['PYTHONHASHSEED'] = str(SEED)
np.random.seed(SEED)
tf.random.set_seed(SEED)

df = pd.read_csv("UFC_TRAIN.csv")

# train/test split
X = df.drop(["Winner","B_fighter","R_fighter"], axis=1).values
y = df["Winner"].values
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2, random_state=SEED)

# scaling
scaler = MinMaxScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

print(f"X_train shape: {X_train.shape} | X_test shape: {X_test.shape} | y_train shape: {y_train.shape} | y_test shape: {y_test.shape}")

# Parameters
MIN = 30
MAX = 256
STEP = 16
MAX_TRIALS = 2
EXE_PER_TRIAL = 1
EPOCHS = 10
PATIENCE = 16

# function to build the model (argument: hyper-parameter)
def build_model(hp):
    # model
    model = Sequential()
    
    # first layer's no. of neurons = hp.Int range of values to test
    model.add(Dense(hp.Int("input_units", min_value=MIN, max_value=MAX, step=STEP), activation='relu'))
    
    # range of 1 to 10 layers to test
    for i in range(hp.Int("no. Of Hidden Layers", 1, 10)):
        # for each added layer, again test range of neurons and add dropout
        model.add(Dense(hp.Int(f"Hidden_layer_{i+1}_units", min_value=MIN, max_value=MAX, step=STEP), activation='relu'))
        model.add(Dropout(0.5))
    
    # output layer
    model.add(Dense(units=1, activation='sigmoid'))

    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

tuner = RandomSearch(
        build_model, # name of the function that builds the model
        objective="val_accuracy", # the thing that we're interested to trace
        max_trials = MAX_TRIALS, # no. of combinations to try
        executions_per_trial = EXE_PER_TRIAL, # no. of times to train each combination (true avg)
        directory=LOG_DIR) # directory to save the outputs

# prevent divergence of loss & val_loss via early stopping
from tensorflow.keras.callbacks import EarlyStopping
early_stop = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=PATIENCE)

X_train shape: (3336, 42) | X_test shape: (835, 42) | y_train shape: (3336,) | y_test shape: (835,)


In [10]:
tuner.search(x=X_train,
             y=y_train,
             epochs = EPOCHS,
#             batch_size = 64,
             callbacks=[early_stop],
             validation_data=(X_test,y_test))

Train on 3336 samples, validate on 835 samples
Epoch 1/10
3336/3336 [==============================] - ETA: 2:44 - loss: 0.7221 - accuracy: 0.46 - ETA: 22s - loss: 0.7012 - accuracy: 0.5179 - ETA: 13s - loss: 0.6818 - accuracy: 0.570 - ETA: 8s - loss: 0.6727 - accuracy: 0.586 - ETA: 6s - loss: 0.6794 - accuracy: 0.57 - ETA: 5s - loss: 0.6774 - accuracy: 0.58 - ETA: 3s - loss: 0.6750 - accuracy: 0.59 - ETA: 3s - loss: 0.6694 - accuracy: 0.61 - ETA: 2s - loss: 0.6697 - accuracy: 0.61 - ETA: 2s - loss: 0.6703 - accuracy: 0.62 - ETA: 1s - loss: 0.6654 - accuracy: 0.62 - ETA: 1s - loss: 0.6635 - accuracy: 0.63 - ETA: 1s - loss: 0.6630 - accuracy: 0.63 - ETA: 0s - loss: 0.6638 - accuracy: 0.63 - ETA: 0s - loss: 0.6628 - accuracy: 0.63 - ETA: 0s - loss: 0.6610 - accuracy: 0.63 - ETA: 0s - loss: 0.6607 - accuracy: 0.63 - ETA: 0s - loss: 0.6600 - accuracy: 0.63 - ETA: 0s - loss: 0.6602 - accuracy: 0.63 - ETA: 0s - loss: 0.6586 - accuracy: 0.63 - 3s 963us/sample - loss: 0.6585 - accuracy: 0.6400

Train on 3336 samples, validate on 835 samples
Epoch 1/10
3336/3336 [==============================] - ETA: 3:03 - loss: 0.6435 - accuracy: 0.75 - ETA: 25s - loss: 0.6570 - accuracy: 0.6607 - ETA: 13s - loss: 0.6384 - accuracy: 0.687 - ETA: 8s - loss: 0.6541 - accuracy: 0.672 - ETA: 6s - loss: 0.6533 - accuracy: 0.66 - ETA: 4s - loss: 0.6492 - accuracy: 0.66 - ETA: 3s - loss: 0.6464 - accuracy: 0.67 - ETA: 3s - loss: 0.6454 - accuracy: 0.67 - ETA: 2s - loss: 0.6429 - accuracy: 0.67 - ETA: 1s - loss: 0.6431 - accuracy: 0.67 - ETA: 1s - loss: 0.6421 - accuracy: 0.67 - ETA: 1s - loss: 0.6405 - accuracy: 0.67 - ETA: 0s - loss: 0.6396 - accuracy: 0.67 - ETA: 0s - loss: 0.6401 - accuracy: 0.67 - ETA: 0s - loss: 0.6394 - accuracy: 0.67 - ETA: 0s - loss: 0.6412 - accuracy: 0.67 - ETA: 0s - loss: 0.6404 - accuracy: 0.67 - 3s 946us/sample - loss: 0.6405 - accuracy: 0.6709 - val_loss: 0.6226 - val_accuracy: 0.7090
Epoch 2/10
3336/3336 [==============================] - ETA: 1s - loss: 0.5986 - ac

INFO:tensorflow:Oracle triggered exit


In [6]:
print(tuner.results_summary()) # top 10 trials
print(tuner.get_best_hyperparameters()[0].values) #values of best hyper-parameters
#print(tuner.get_best_models()[0].summary())

None
{'input_units': 158, 'no. Of Hidden Layers': 2, 'Hidden_layer_1_units': 62, 'Hidden_layer_2_units': 30}
